In [ ]:
#!pip install tflearn

     |████████████████████████████████| 107 kB 5.2 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=7bbaaaefeef76794f6b0e60a7898567155ff8acc1e122928d4f583fe9506f631
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random

import json

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
import json 
import csv 
from google.colab import files
uploaded = files.upload()

Saving intents.json to intents.json


In [ ]:
import pandas as pd


In [ ]:
with open('intents.json') as file:
    data = json.load(file)

In [ ]:
words = []
labels = []
docs_x = []
docs_y = []

In [ ]:
#nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [ ]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [ ]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [ ]:
training = numpy.array(training)
output = numpy.array(output)

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [ ]:

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 4999  | total loss: 0.02110 | time: 0.016s
| Adam | epoch: 1000 | loss: 0.02110 - acc: 1.0000 -- iter: 32/35
Training Step: 5000  | total loss: 0.02119 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.02119 - acc: 1.0000 -- iter: 35/35
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hello
Hello!
You: whats up
Hello!
You: what is your name
My name is CZAR!
You: see you later
Goodbye
You: whats your age
I am tweenty three years old
You: age?
I am tweenty three years old
You: how old are you
tweny three
You: name?
you can call me CZAR!
You: whats ya name
I am CZAR assistant of DISHANT
You: what are your products
There is a lot of projects available on Kaggle, Github and Medium to start with!
You: recommend something to me
There is a lot of projects available on Kaggle, Github and Medium to start with!
You: hours of open
We are open 
You: opening hours
We are open 
You: opening hours
We are open 
You: what are your hours
anytime! we have all service
You: quit
